## Fully Connected layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
t_max = 2**8
dims = (1,1,1,1)
n_outputs = 3

np.random.seed(seed=19)
np.set_printoptions(suppress=True)
weights = (np.random.rand(n_outputs,np.product(dims[1:])) - 0.5) * 2 # np.ones((n_outputs,np.product(dims[1:]))) # 
biases = np.zeros((n_outputs)) # (np.random.rand(n_outputs) - 0.5) / 2 # 

loihi_model = quartz.Network([
    layers.InputLayer(dims=dims[1:]),
    layers.Dense(weights=weights, biases=None, rectifying=True), # careful NO BIAS
    layers.MonitorLayer(),
])

# input0 = quartz.probe(loihi_model.layers[0].blocks[-1])
inputs = [quartz.probe(block) for block in loihi_model.layers[0].blocks[1:]]
relcos = [quartz.probe(block) for block in loihi_model.layers[1].blocks[1:]]
calcs = [quartz.probe(block.neurons[0]) for block in loihi_model.layers[1].blocks[1:]]
trigger0 = quartz.probe(loihi_model.layers[0].blocks[0])
trigger1 = quartz.probe(loihi_model.layers[1].blocks[0])
outputs = quartz.probe(loihi_model.layers[-1])

values = np.random.rand(*dims) / 3 # np.ones((np.product(dims))) * 0.5 # np.zeros((np.product(dims))) # 
quantized_values = (values*t_max).round()/t_max
weight_acc = loihi_model.layers[1].weight_acc
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

pt_model = nn.Sequential(
    nn.Linear(in_features=np.product(dims[1:]), out_features=n_outputs), 
    nn.ReLU()
)
pt_model[0].weight = torch.nn.Parameter(torch.tensor(quantized_weights))
pt_model[0].bias = torch.nn.Parameter(torch.tensor((quantized_biases)))
model_output = pt_model(torch.tensor(quantized_values.reshape(dims[0], -1))).detach().numpy()
model_output

In [ ]:
loihi_model(values, t_max, steps_per_image=1600, partition='nahuku32', profiling=False)

In [ ]:
capture = [relco.plot() for relco in relcos]

In [ ]:
trigger1.target.get_connected_blocks()

In [ ]:
loihi_model.data

In [ ]:
relcos[0].output()

In [ ]:
loihi_model.layers[1].blocks[1].neurons[0].synapses

In [ ]:
loihi_model.layers[1].weight_e

In [ ]:
import quartz
import numpy as np
values = np.random.rand(1,1,10,1)
values[0][0][0] = 0
values[0][0][1] = 1
# values[1][0][0] = 0
# values[1][0][1] = 1
spikes = quartz.decode_values_into_spike_input(values, t_max=2**7, steps_per_sample=500)
print(values)
print(spikes)

In [ ]:
output.EnergyPhase(1)

In [ ]:
sum(abs(weights[0,:]))

In [ ]:
output_probe.target.blocks[0].monitor

In [ ]:
loihi_output

In [ ]:
ok = [relco.plot() for relco in relcos]

In [ ]:
ok = [calc.plot() for calc in calcs]

## Convolutional Layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

t_max = 2**8
input_dims = (1,6,5,5)
weight_dims = (6,2,5,5)
groups = 3
kernel_size = weight_dims[2:]
np.random.seed(seed=27)
weights = (np.random.rand(*weight_dims)-0.5) / 3 # np.ones((weight_dims)) # 
biases = (np.random.rand(weight_dims[0])-0.5) / 2 # np.zeros((weight_dims[0])) #

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims[1:]),
    layers.Conv2D(weights=weights, biases=biases, groups=groups),
    layers.MonitorLayer(),
])

input0 = quartz.probe(loihi_model.layers[0].blocks[0])
relcos = [quartz.probe(block) for block in loihi_model.layers[1].blocks]

values = np.random.rand(*input_dims) / 2 # np.zeros((np.product(input_dims)))
quantized_values = (values*t_max).round()/t_max
weight_acc = loihi_model.layers[1].weight_acc
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(nn.Conv2d(in_channels=weight_dims[1]*groups, out_channels=weight_dims[0], kernel_size=kernel_size, groups=groups), nn.ReLU())
model[0].weight = torch.nn.Parameter(torch.tensor(quantized_weights))
model[0].bias = torch.nn.Parameter(torch.tensor((quantized_biases)))
model_output = model(torch.tensor(quantized_values)).detach().numpy()
model_output[0]

In [ ]:
loihi_output = loihi_model(values, t_max, steps_per_image=1600, partition='nahuku32')
loihi_output[0]

In [ ]:
%debug

In [ ]:
ok = [relco.plot() for relco in relcos]

## Max Pooling layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

t_max = 2**8
input_dims = (1,16,10,10)
kernel_size = [2,2]

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims[1:]),
    layers.MaxPool2D(kernel_size=kernel_size),
    layers.MonitorLayer(),
])

#pools = [quartz.probe(block) for block in loihi_model.layers[1].blocks]

np.random.seed(seed=45)
values = np.random.rand(*input_dims)
quantized_values = (values*t_max).round()/t_max

model = nn.Sequential(nn.MaxPool2d(kernel_size=kernel_size, stride=kernel_size[0]), nn.ReLU())
model_output = model(torch.tensor(quantized_values)).squeeze().detach().numpy()
#model_output

In [ ]:
loihi_output = loihi_model(values, t_max, logging=False, partition="loihi_2h") # , steps_per_image=2000)

In [ ]:
loihi_model

In [ ]:
loihi_model.print_core_layout()

In [ ]:
loihi_model.layers[0].n_incoming_connections()

In [ ]:
loihi_model.layers[0].n_recurrent_connections()

In [ ]:
loihi_model.layers[1].n_incoming_connections()

In [ ]:
loihi_model.layers[0].blocks[0].neurons[1].synapses

## Convpool layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

input_dims = ( 2,4,4)
weight_dims = ( 5,2,3,3)
t_max = 2**8
batch_size = 400
conv_kernel_size = weight_dims[2:]
pooling_kernel_size = [2,2]
pooling_stride = 2

np.random.seed(seed=47)
np.set_printoptions(suppress=True)
weights = (np.random.rand(*weight_dims)-0.5) / 5 # np.zeros(weight_dims) #
biases = (np.random.rand(weight_dims[0])-0.5) # np.zeros(weight_dims[0]) # 

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.ConvPool2D(weights=weights, biases=biases, pool_kernel_size=pooling_kernel_size),
    layers.MonitorLayer(),
])
trigger0 = quartz.probe(loihi_model.layers[0].blocks[-1])
trigger1 = quartz.probe(loihi_model.layers[1].blocks[0])
convmaxs1 = [quartz.probe(block) for block in loihi_model.layers[1].blocks[11:]]

loihi_model1 = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.Conv2D(weights=weights, biases=biases),
    layers.MaxPool2D(kernel_size=pooling_kernel_size, stride=pooling_stride),
    layers.MonitorLayer(),
])
maxpools = [quartz.probe(block) for block in loihi_model1.layers[2].blocks]

values = np.random.rand(batch_size, *input_dims) / 4
quantized_values = (values*t_max).round()/t_max
weight_acc = loihi_model.layers[1].weight_acc
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(
    nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=conv_kernel_size), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size, stride=pooling_stride), nn.ReLU(),
)
model[0].weight = nn.Parameter(torch.tensor(weights))
model[0].bias = nn.Parameter(torch.tensor(biases))
model_output = model(torch.tensor(values)).detach().numpy()
#model_output

In [ ]:
loihi_output = loihi_model(values, t_max, steps_per_image=2048, logging=True)
#loihi_output

In [ ]:
loihi_output1 = loihi_model1(values, t_max)
loihi_output1

In [ ]:
ok = [convmax.plot() for convmax in convmaxs1]